In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandarallel import pandarallel
import numpy as np
import matplotlib.pyplot as plt

from microsim.population import NHANESDirectSamplePopulation
from microsim.statsmodel_linear_risk_factor_model import StatsModelLinearRiskFactorModel
from microsim.regression_model import RegressionModel
from microsim.data_loader import load_model_spec
from microsim.outcome import OutcomeType, Outcome
from microsim.outcome_details.stroke_details import StrokeSubtypeModelRepository, StrokeNihssModel, StrokeTypeModel
from microsim.stroke_outcome import StrokeOutcome, StrokeSubtype, StrokeType, Localization
from microsim.treatment_strategy_repository import TreatmentStrategyRepository
from microsim.bp_treatment_strategies import AddASingleBPMedTreatmentStrategy, AddNBPMedsTreatmentStrategy
from microsim.treatment import TreatmentStrategiesType, TreatmentStrategyStatus
from microsim.outcome import OutcomeType
from microsim.stroke_partition_model import StrokePartitionModel

import scipy.special as scipySpecial

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

import time

timeI = time.time()

popSize = 400000
nYears = 5
bpMedsAdded = 2

In [2]:
treatmentStrategies = TreatmentStrategyRepository()
#treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddASingleBPMedTreatmentStrategy()
treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddNBPMedsTreatmentStrategy(bpMedsAdded)

In [3]:
%%time
controlPop = NHANESDirectSamplePopulation(popSize, 1999)

CPU times: user 2min 31s, sys: 821 ms, total: 2min 32s
Wall time: 2min 32s


In [4]:
%%time
controlPop.advance_parallel(nYears, None, 5)

CPU times: user 1min 12s, sys: 35 s, total: 1min 47s
Wall time: 9min 18s


In [5]:
%%time
treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)

CPU times: user 2min 55s, sys: 586 ms, total: 2min 55s
Wall time: 2min 55s


In [6]:
%%time
treatmentPop.advance_parallel(1, treatmentStrategies = treatmentStrategies, nWorkers=5)
treatmentStrategies._repository[TreatmentStrategiesType.BP.value].status = TreatmentStrategyStatus.MAINTAIN
treatmentPop.advance_parallel(nYears-1, treatmentStrategies = treatmentStrategies, nWorkers=5)

CPU times: user 3min 36s, sys: 7min 47s, total: 11min 23s
Wall time: 22min 14s


In [7]:
controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                           controlPop._people)))/controlPop._n
treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
strokeRR = treatmentRisk/controlRisk

controlRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                           controlPop._people)))/controlPop._n
treatmentRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                             treatmentPop._people)))/treatmentPop._n
miRR = treatmentRiskMI/controlRiskMI

In [8]:
strokeRR, miRR

(0.5931825422108952, 0.7717015653588128)

In [9]:
from microsim.cv_model import CVModelMale, CVModelFemale
import importlib.util
moduleCV = importlib.import_module("microsim.cv_model_repository")
moduleStroke = importlib.import_module("microsim.stroke_partition_model_repository")

In [10]:
##19 -11.8284175 -12.9715475 0.00390625 -2.4164418087083006 0.001953125 0.7823834196891192 0.8673245614035087

#interceptStep = 0.5
interceptStep = 0.01
interceptChange = 0
#interceptMale = -11.679980
interceptMale = -11.9312500
#interceptFemale = -12.823110
interceptFemale = -13.051250

#strokeInterceptStep = 0.25
strokeInterceptStep = 0.01
strokeInterceptChange = 0
model_spec = load_model_spec("StrokeMIPartitionModel")
#strokeIntercept = model_spec["coefficients"]["Intercept"]
strokeIntercept = -2.6159375

strokeRRRef = 0.79**bpMedsAdded
miRRRef = 0.87**bpMedsAdded

In [11]:
%%time
for i in range(20):

    strokeRRDiff = strokeRR-strokeRRRef
    miRRDiff = miRR-miRRRef
    
    if (strokeRRDiff>0) & (miRRDiff>0):
        interceptChange = -interceptStep
    elif (strokeRRDiff<0) & (miRRDiff<0):
        interceptChange = interceptStep
    else:
        interceptChange = 0
    interceptMale += interceptChange
    interceptFemale += interceptChange

    if strokeRRDiff>0:
        strokeInterceptChange = -strokeInterceptStep
    else:
        strokeInterceptChange = strokeInterceptStep
    strokeIntercept += strokeInterceptChange
    
    print(i, interceptMale, interceptFemale, interceptChange, strokeIntercept, strokeInterceptChange, 
              strokeRR, miRR)    
    
    #class CVModelMale(CVModelBase):
    #    """CV model details for male gender."""
    #    def __init__(self):
    #        maleCVCoefficients = {
    #        "lagAge": 0.064200,
    #        "black": 0.482835,
    #        "lagSbp#lagSbp": -0.000061,
    #        "lagSbp": 0.038950,
    #        "current_bp_treatment": 2.055533,
    #        "current_diabetes": 0.842209,
    #        "current_smoker": 0.895589,
    #        "lagAge#black": 0,
    #        "lagSbp#current_bp_treatment": -0.014207,
    #        "lagSbp#black": 0.011609,
    #        "black#current_bp_treatment": -0.119460,
    #        "lagAge#lagSbp": 0.000025,
    #        "black#current_diabetes": -0.077214,
    #        "black#current_smoker": -0.226771,
    #        "lagSbp#black#current_bp_treatment": 0.004190,
    #        "lagAge#lagSbp#black": -0.000199,
    #        "Intercept": interceptMale,
    #        }
    #        tot_chol_hdl_ratio=0.193307
    #        black_race_x_tot_chol_hdl_ratio=-0.117749
    #        super().__init__(maleCVCoefficients, tot_chol_hdl_ratio, black_race_x_tot_chol_hdl_ratio)

    #class CVModelFemale(CVModelBase):
    #    """CV model details for female gender."""
    #    def __init__(self):
    #        femaleCVCoefficients = {
    #        "lagAge": 0.106501,
    #        "black": 0.432440,
    #        "lagSbp#lagSbp": 0.000056,
    #        "lagSbp": 0.017666,
    #        "current_bp_treatment": 0.731678,
    #        "current_diabetes": 0.943970,
    #        "current_smoker": 1.009790,
    #        "lagAge#black": -0.008580,
    #        "lagSbp#current_bp_treatment": -0.003647,
    #        "lagSbp#black": 0.006208,
    #        "black#current_bp_treatment": 0.152968,
    #        "lagAge#lagSbp": -0.000153,
    #        "black#current_diabetes": 0.115232,
    #        "black#current_smoker": -0.092231,
    #        "lagSbp#black#current_bp_treatment": -0.000173,
    #        "lagAge#lagSbp#black": -0.000094,
    #        "Intercept": interceptFemale,
    #        }
    #        tot_chol_hdl_ratio=0.151318
    #        black_race_x_tot_chol_hdl_ratio=0.070498
    #        super().__init__(femaleCVCoefficients, tot_chol_hdl_ratio, black_race_x_tot_chol_hdl_ratio)
  
    class CVModelMaleFor2bpMedsAdded(CVModelMale):
        def __init__(self):
            super().__init__(intercept = interceptMale)

    class CVModelFemaleFor2bpMedsAdded(CVModelFemale):
        def __init__(self):
            super().__init__(intercept = interceptFemale)

    setattr(moduleCV, "CVModelFemaleFor2bpMedsAdded", CVModelFemaleFor2bpMedsAdded)
    setattr(moduleCV, "CVModelMaleFor2bpMedsAdded", CVModelMaleFor2bpMedsAdded)
    
    #class StrokePartitionModel(StatsModelLinearRiskFactorModel):
    #    """Fatal stroke probability estimated from our meta-analysis of BASIC, NoMAS, GCNKSS, REGARDS."""

    #    def __init__(self):
    #        model_spec = load_model_spec("StrokeMIPartitionModel")
    #        model_spec["coefficients"]["Intercept"] = strokeIntercept
    #        super().__init__(RegressionModel(**model_spec))
    #        self._stroke_case_fatality = 0.15
    #        self._stroke_secondary_case_fatality = 0.15
   
    #    def will_have_fatal_stroke(self, person):
    #        fatalStrokeProb = self._stroke_case_fatality
    #        fatalProb = self._stroke_secondary_case_fatality if person._stroke else fatalStrokeProb
    #        return person._rng.uniform(size=1) < fatalProb

    #    def get_next_stroke_probability(self, person):
    #        #Q: I am not sure why it was set to 0 at the beginning
    #        strokeProbability = 0
    #        strokeProbability = scipySpecial.expit( super().estimate_next_risk(person) )
    #        return strokeProbability
    
    #    def generate_next_outcome(self, person):
    #        fatal = self.will_have_fatal_stroke(person)
    #        nihss = StrokeNihssModel().estimate_next_risk(person)
    #        strokeSubtype = StrokeSubtypeModelRepository().get_stroke_subtype(person)
    #        strokeType = StrokeTypeModel().get_stroke_type(person)
    #        #localization = Localization.LEFT_HEMISPHERE
    #        #disability = 3 
    #        #return StrokeOutcome(fatal, nihss, strokeType, strokeSubtype, localization, disability)
    #        return StrokeOutcome(fatal, nihss, strokeType, strokeSubtype)
        
    #    def update_cv_outcome(self, person, fatal):
    #        person._outcomes[OutcomeType.CARDIOVASCULAR][-1][1].fatal = fatal
        
    #    def get_next_outcome(self, person):
    #        if person.has_outcome_at_current_age(OutcomeType.CARDIOVASCULAR):
    #            if person._rng.uniform(size=1) < self.get_next_stroke_probability(person):
    #                strokeOutcome = self.generate_next_outcome(person)
    #                #if we decide to use different stroke/mi models, double check if fatality needs to be decided in CVModel
    #                self.update_cv_outcome(person, strokeOutcome.fatal)
    #                return strokeOutcome
    #            else: 
    #                return None
    class StrokePartitionModelFor2bpMedsAdded(StrokePartitionModel):
        def __init__(self):
            intercept = strokeIntercept
            super().__init__(intercept=intercept)
            
    setattr(moduleStroke, "StrokePartitionModelFor2bpMedsAdded", StrokePartitionModelFor2bpMedsAdded)
    
    treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)
    treatmentStrategies._repository[TreatmentStrategiesType.BP.value].status = TreatmentStrategyStatus.BEGIN
    treatmentPop.advance(1, treatmentStrategies = treatmentStrategies)
    treatmentStrategies._repository[TreatmentStrategiesType.BP.value].status = TreatmentStrategyStatus.MAINTAIN
    treatmentPop.advance(nYears-1, treatmentStrategies = treatmentStrategies)
     
    treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
    strokeRR = treatmentRisk/controlRisk
    
    treatmentRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                                 treatmentPop._people)))/treatmentPop._n
    miRR = treatmentRiskMI/controlRiskMI
    
    if ((strokeRR-strokeRRRef)*strokeRRDiff<0) & ((miRR-miRRRef)*miRRDiff<0):
        interceptStep = interceptStep/2
        strokeInterceptStep = strokeInterceptStep/2

0 -11.93125 -13.05125 0 -2.6059375 0.01 0.5931825422108952 0.7717015653588128
1 -11.92125 -13.04125 0.01 -2.5959375000000002 0.01 0.5734310289901242 0.7452734295588535
2 -11.92125 -13.04125 0 -2.5859375000000004 0.01 0.6129340554316661 0.7615368977434438
3 -11.92125 -13.04125 0 -2.5759375000000007 0.01 0.5839439311882765 0.7623500711526733
4 -11.91125 -13.03125 0.01 -2.565937500000001 0.01 0.5702453010512901 0.7505590567188454
5 -11.91125 -13.03125 0 -2.555937500000001 0.01 0.6158012105766167 0.7796300060988006
6 -11.901250000000001 -13.02125 0.01 -2.5459375000000013 0.01 0.6119783370500159 0.7554380971742224
7 -11.891250000000001 -13.01125 0.01 -2.5359375000000015 0.01 0.6164383561643835 0.7529985769465338
8 -11.891250000000001 -13.01125 0 -2.5259375000000017 0.01 0.6193055113093341 0.7834925797926408
9 -11.891250000000001 -13.01125 0 -2.515937500000002 0.01 0.6119783370500159 0.7731246188249645
10 -11.901250000000001 -13.02125 -0.01 -2.5259375000000017 -0.01 0.6463841987894233 0.7662

In [12]:
timeF = time.time()
(timeF-timeI)/60

821.1230021834374

In [ ]:
strokeRRDiff = strokeRR-strokeRRRef
if strokeRRDiff>0:
    interceptChange = -interceptStep
else:
    interceptChange= interceptStep
interceptMale += interceptChange
interceptFemale += interceptChange

print(interceptMale, interceptFemale, interceptStep, interceptChange, strokeRRDiff, strokeRR)

In [ ]:
#19 -11.8284175 -12.9715475 0.00390625 -2.4164418087083006 0.001953125 0.7823834196891192 0.8673245614035087
interceptMale = -11.8469721875
interceptFemale = -12.9901021875
interceptChange = -0.00048828125

In [ ]:
controlRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                           controlPop._people)))/controlPop._n
treatmentRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                             treatmentPop._people)))/treatmentPop._n
miRR = treatmentRiskMI/controlRiskMI
miRR

In [ ]:
#RESULTS
#1 bp: -11.795917500000009 -12.939047500000008 0 -2.400191808699997 -0.00125 0.7918781725888325 0.8565955117679255
#2 bp -11.931250000000002 -13.051250000000003 0 -2.6159375 0.0015625 0.6071428571428572 0.7655677655677655
#2 bp: 15 -11.911249999999999 -13.031249999999998 0 -2.5334375000000016 0.0025 0.6126154826377828 0.7658060581418988
#3 bp: -12.007714375000003 -13.150500000000003 -0.0125 -2.6649999999999987 -0.0125 0.5244482907832108 0.6721311475409835
#4 bp: -12.085210000000002 -13.228000000000002 0 -2.7462 0.01 0.38026981450252956 0.5751421608448416

In [ ]:
%%time
treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)

In [ ]:
%%time
treatmentPop.advance(nYears, treatmentStrategies = treatmentStrategies)

In [ ]:
controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                           controlPop._people)))/controlPop._n
treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
strokeRR = treatmentRisk/controlRisk
strokeRR

In [ ]:
if (strokeRR-strokeRRRef)*strokeRRDiff<0:
    interceptStep = interceptStep/2
strokeRRDiff = strokeRR-strokeRRRef

In [ ]:
test = {"male": {"1": "one",
                 "2": "two"},
        "female": {"1": "one",
                 "2": "two"}}

In [ ]:
test["male"]["2"]